<a href="https://colab.research.google.com/github/fernandanlisboa/technologies_clf/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento



*   Davi Costa
*   Fernanda Lisboa
*   João Felipe
*   Maria Antônia





## Importações

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy

In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Dataset
##### O dataset utilizado foi disponibilizado pelo professor Braian, e contém pequenos textos retirados de colunas de reportagens ou sites informativos sobre linguagens de programação.

In [ ]:
drive_path = '/content/drive'
directory = f'{drive_path}/MyDrive/2023.2/Tópicos em Inteligência/Dasafio 01 - Classificação/Entrega_Desafio_1'
#directory = f'{drive_path}/MyDrive/Dasafio 01 - Classificação'
csv_file = 'base_textos_stack.csv'

In [ ]:
#Conexão com diretório do google drive
drive.mount(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Leitura do dataset
df = pd.read_csv(f'{directory}/base_textos_stack.csv')

In [ ]:
df.head()

,excerpt,classification
0,Software-defined networking (SDN),1
1,SPDY is an experimental protocol for multiplex...,1
2,ASP.NET SignalR is a library for ASP.NET devel...,0
3,Apache Kafka is a publish-subscribe messaging ...,2
4,Kubernetes is an open source implementation of...,1


## Limpeza do Texto

In [ ]:
#Função para trocar caracteres especiais em nomes reservados
def replacer(text):
  return text.replace("C#","csharp").replace(".NET","dotNET").replace("F#","Fsharp")

In [ ]:
#Regex para retirar os links
link_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [ ]:
# Função para Limapr a base
def clean_text(text):
    text = replacer(text)
    text_without_links = re.sub(link_pattern, '', text)

    # Substitui espaços duplicados por um único espaço
    cleaned_text = re.sub(r' +', ' ', text_without_links)
    # Troca letras maiúsculas por minúsculas
    lower_text = cleaned_text.lower()

    # Garante que não haja espaços no início ou no final da string
    return lower_text.strip()

Após análisar o dataset a equipe observou que os números não apresentam significância para a lemmatização e nem para o treino do modelo, sendo assim, foram todos os números foram removidos do dataset.

In [ ]:
# Limpando a base
df['excerpt_parsed'] = [clean_text(content) for content in df['excerpt']]

In [ ]:
print('Antes da limpeza:')
example = df.loc[2]
print(example['excerpt'])

print('\n\nDepois da limpeza:')
print(example['excerpt_parsed'])

Antes da limpeza:
ASP.NET SignalR is a library for ASP.NET developers that makes it incredibly simple to add real-time web functionality to your applications. What is "real-time web" functionality? It's the ability to have your server-side code push content to the connected clients as it happens, in real-time.


Depois da limpeza:
aspdotnet signalr is a library for aspdotnet developers that makes it incredibly simple to add real-time web functionality to your applications. what is "real-time web" functionality? it's the ability to have your server-side code push content to the connected clients as it happens, in real-time.


## *Stop Words*

In [ ]:
# Baixando as listas de stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Carregando a lista de stopwords em inglês
stop_words = list(stopwords.words('english'))
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Lematização

**Stemming**: Este método tenta cortar o sufixo (e em alguns casos o prefixo) das palavras para obter a raiz. Por exemplo, o stem (raiz) da palavra "correndo" pode ser "corr". É geralmente mais rudimentar e pode não resultar sempre em palavras que têm um significado claro por si mesmas. Por exemplo, "universidade" e "universal" podem ser reduzidas ao mesmo stem "univers". Tende a ser mais rápido e menos complexo, pois usa algoritmos heurísticos para cortar os sufixos das palavras.



**Lematização**: Este método tenta reduzir a palavra à sua forma base ou dicionário usando um vocabulário e análise morfológica da palavra. Por exemplo, a lema da palavra "correndo" seria "correr". Considera o contexto e converte a palavra para o seu significado base, o que é conhecido como lema. É mais preciso que o stemming e geralmente resulta em palavras que estão presentes no dicionário. Pode ser mais lento e requer mais recursos, pois considera o contexto gramatical da palavra e exige um vocabulário e uma análise morfológica. Em algumas línguas, como o inglês, isso pode envolver considerações adicionais, como determinar a parte da fala de uma palavra no contexto.

In [ ]:
# WordNet é uma grande base de dados léxica da língua inglesa
nltk.download('wordnet')

# Punkt é um modelo usado pelo NLTK para tokenizar sentenças.
nltk.download('punkt')

# Ele é usado para determinar a classe gramatical de cada palavra em uma sentença (por exemplo, substantivo, verbo, adjetivo, etc.).
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Retorna o token da palavra que não seja uma stop word
func_lematizator = lambda x: " ".join([w.lemma_ for w in nlp(x) if w.text not in stop_words])

df['excerpt_lem'] = df['excerpt_parsed'].apply(func_lematizator)

In [ ]:
print('Antes da lematização:')
print(df['excerpt_parsed'])

print('\n\nDepois da lematização:')
print(df['excerpt_lem'])

Antes da lematização:
0                      software-defined networking (sdn)
1      spdy is an experimental protocol for multiplex...
2      aspdotnet signalr is a library for aspdotnet d...
3      apache kafka is a publish-subscribe messaging ...
4      kubernetes is an open source implementation of...
                             ...                        
262    docker provides a high-level api to containeri...
263    axon is a java application framework for imple...
264    html5 (hyper text markup language, version 5) ...
265    csharp 4.0 introduced features like dynamic ob...
266    nancy is a lightweight web framework for the ....
Name: excerpt_parsed, Length: 267, dtype: object


Depois da lematização:
0                   software - define networking ( sdn )
1      spdy experimental protocol multiplexe multiple...
2      aspdotnet signalr library aspdotnet developer ...
3      apache kafka publish - subscribe messaging fra...
4      kubernete open source implementation conta

## Exportando Resultados

In [ ]:
#Função para retirar os caracteres especiais
def clean_especial_chars(text):
    text_without_special_chars = re.sub(r'[^a-zA-Z ]+', '', text).lower().strip()

    return text_without_special_chars

In [ ]:
df['excerpt_lem'] = [clean_especial_chars(content) for content in df['excerpt_lem']]

In [ ]:
# Exportando CSV pré processado
df = df.drop(['excerpt', 'excerpt_parsed'], axis=1)
df.to_csv(f'{directory}/base_textos_pre_processada.csv', sep='|', index=False)